In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pytorch_lightning
!pip install transformers
!pip install sentencepiece

In [ ]:
!pip install wandb -qqq

In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: beifa (use `wandb login --relogin` to force relogin)


True

In [ ]:
import json
import torch
import datetime
from tqdm import tqdm
import torch.nn as nn
import os, glob, re
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import (T5ForConditionalGeneration,
                          AdamW,
                          T5TokenizerFast as token)

from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
pl.seed_everything(13)
print(torch.__version__)
PATH = '/content/drive/MyDrive/Coleridge_Initiative/input'

Global seed set to 13


1.8.1+cu101


In [ ]:
!nvidia-smi

Tue Jun 15 18:33:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Model

In [ ]:
# df = df[['id', 'text', 'answer']]
# df.columns = ['id', 'text', 'summary']
# df.to_csv('/content/drive/MyDrive/Coleridge_Initiative/input/v6_data_summary.csv', index= False)

df = pd.read_csv('/content/drive/MyDrive/Coleridge_Initiative/input/v6_data_summary.csv')
df.head()

,id,text,summary
0,40bf999d-8537-47f7-8913-bd67b060146d,the quality of educational programs has been a...,trends in international mathematics and scienc...
1,40bf999d-8537-47f7-8913-bd67b060146d,knowledge about the potential usefulness of th...,conclusions
2,5631ebad-191f-44d7-9c21-89b4f84852dc,as mentioned above the sars cov 2 s glycoprote...,genome sequence of sars cov 2
3,43f2034e-9220-4103-84bf-79f67b75f29c,attributed to a cold core anticyclonic eddy ra...,journal of geophysical research oceans
4,43f2034e-9220-4103-84bf-79f67b75f29c,deep arctic quality profiles between and from ...,world ocean database


In [ ]:
BATCH = 3
EPOCHS = 6
FRAC = 0.9

config={
    'type_model': 'summary',
    "learning_rate": 0.0003,
    "architecture": "T5",
    'model': 't5-small', #'t5-base', # t5-small
    "dataset": "Coleridge Initiative ",
    'tex_max_len': 512,
    'summar_max_len': 18,
    'batch_size' : BATCH,
    'epoch':EPOCHS,
    'device': 'cuda'
}


wandb.init(project="ci",
           config=config,
           name =  f"{config['architecture']}_{config['model']}_{config['type_model']}"+\
                   f"_LR:{config['learning_rate']}_BATCH:{BATCH}_FRAC_DATA:{FRAC}"
           )
config_w = wandb.config

In [ ]:
class CI_Dataset(Dataset):
    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: token,
        config
    ):

        self.tokenizer = tokenizer
        self.data = data        
        self.tex_max_len = config['tex_max_len']
        self.summar_max_len = config['summar_max_len']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index:int):
        txt = self.data.iloc[index]

        encode_txt = token(            
            txt['text'],
            max_length = self.tex_max_len, 
            padding= 'max_length',
            truncation = True, #'only_second',
            return_attention_mask = True,
            add_special_tokens =True,
            return_tensors = 'pt'
            )
        
        encode_summary = token( 
            txt['summary'],
            max_length = self.summar_max_len,
            padding= 'max_length',
            truncation = True,
            return_attention_mask = True,
            add_special_tokens =True,
            return_tensors = 'pt'
            )
        
        labels = encode_summary['input_ids']
        labels[labels == 0] = -100

        return dict(          
            text = txt['text'],
            summary = txt['summary'],
            input_ids = encode_txt['input_ids'].flatten(),
            attention_mask = encode_txt['attention_mask'].flatten(),
            labels = labels.flatten(),
            label_attention_mask = encode_summary['attention_mask'].flatten(),

            )
        
class CI(nn.Module):

    def __init__(self, config):
        super().__init__()   
        self.model = T5ForConditionalGeneration.from_pretrained(config['model'], return_dict = True)

    def forward(self, input_ids, attention_mask, summary_attention_mask, labels = None):
        out = self.model(input_ids = input_ids,
                         attention_mask = attention_mask,
                         labels = labels,
                         decoder_attention_mask = summary_attention_mask
                         )
        return out.loss, out.logits

In [ ]:
MODEL = config['model']
token = token.from_pretrained(MODEL)
exampe_dataset = CI_Dataset(df, token, config)

for data in exampe_dataset:
    print(data['text'])
    print(data['summary'])
    print(data['input_ids'][:10])
    print(data['attention_mask'][:10]) 
    print(data['label_attention_mask'][:10])  
    break

the quality of educational programs has been an object of debate and research around the world initiatives such as the program for international student assessment pisa and the trends in international mathematics and science study timss show that international organizations such as the organization for economic co operation and development oecd and the international association for the evaluation of educational achievement iea are trying to verify whether schools are adequately preparing their students by comparing their performances aiming to highlight the strengths and weaknesses among the educational systems of different countries higher education has also been the object of quality evaluations around the world ursin huusko aittola kiviniemi muhonen van kemenade pupius hardjono governmental and non governmental organizations have developed ways to certify institutional quality through evaluation or accreditation processes examples of these organizations include the european associat

In [ ]:
# text_count, summary_count = [], []
# for _, row in df.iterrows():
#     txt_count = len(token.encode(row['text']))
#     text_count.append(txt_count)
#     sm_count = len(token.encode(row['summary']))
#     summary_count.append(sm_count)

In [ ]:
# fig, ax = plt.subplots(1, 2)
# ax[0].hist(text_count, bins=100)
# ax[0].set_title('txt count token')
# ax[1].hist(summary_count, bins=20)
# ax[1].set_title('summary count token');

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


def make_current_date()->str:
    x = datetime.datetime.now()
    x = str(x.date())
    return x.replace('-', '_')


def loss_func(pred, target, config):
    tmp = []
    MODEL = config['model']
    tokenizer = token.from_pretrained(MODEL)
    for i in range(len(pred)):
        out_decode = tokenizer.decode(np.argmax(pred[i], axis = 1),
                                  skip_special_tokens=True, 
                                  clean_up_tokenization_spaces=True)       
        tmp.append(jaccard(''.join(out_decode), target[i]))
    return tmp


def train(model, data_loader, optimizer, config, scheduler = None):
    model.train()
    o = []
    for _, txt in enumerate(data_loader):        
        input_ids=txt['input_ids'].to(config['device'])
        mask=txt['attention_mask'].to(config['device'])
        labels = txt['labels'].to(config['device'])
        labels_mask = txt['label_attention_mask'].to(config['device'])
        optimizer.zero_grad()
        loss, out = model(input_ids, mask, labels_mask, labels)
        # print(loss)
        # print('----')    
        # loss = model(input_ids, mask, labels)
        o.append(loss.cpu().detach().numpy())        
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
    return np.mean(o)


def valid(model, data_loader,config):
    model.eval()
    losses = []
    asw = []
    loss_2 = []
    outs = []
    for _, txt in enumerate(data_loader):
        input_ids=txt['input_ids'].to(config['device'])
        mask=txt['attention_mask'].to(config['device'])
        labels = txt['labels'].to(config['device']) 
        labels_mask = txt['label_attention_mask'].to(config['device'])
        answer=txt['summary']

        loss, out = model(input_ids, mask, labels_mask, labels)
        out = out.cpu().detach().numpy()
        loss2 = loss_func(out, answer, config)
        losses.append(loss.cpu().detach().numpy())
        loss_2.append(loss2)
        asw.append(answer)
        outs.append(out)
    # print(losses)
    # print(loss_2)
    return losses, np.vstack(asw), loss_2, np.vstack(outs)


def run(config):
    MODEL = config['model']
    df = pd.read_csv('/content/drive/MyDrive/Coleridge_Initiative/input/v6_data_summary.csv')
    df_small = df.drop_duplicates(subset=['id']).sample(frac = FRAC, random_state = 13).reset_index(drop=True)
    train_df, val_df = train_test_split(df_small, random_state = 13, test_size = 0.1)
    tr = train_df.reset_index(drop=True)
    vl = val_df.reset_index(drop=True)

    tokenizer = token.from_pretrained(MODEL)

    tr_dataset = CI_Dataset(tr,tokenizer,config)
    vl_dataset = CI_Dataset(vl,tokenizer,config)

    tr_loader = DataLoader(tr_dataset, batch_size = config['batch_size'], shuffle = True, num_workers = 4)
    vl_loader = DataLoader(vl_dataset, batch_size = 1, num_workers = 4)

    model = CI(config).to(config['device'])

    optimizer = AdamW(model.parameters(), lr = config['learning_rate'])
    bar =  tqdm(range(config['epoch']))
    wandb.watch(model) 
    best_jaccard = 0
    for e in bar:          
        tl = train(model, tr_loader, optimizer, config)      
        l, _, j, _ = valid(model, vl_loader, config)    
        jaccard = np.mean(j)
        bar.set_description(f'Valid loss: {np.mean(l)}, Jaccard Loss: {jaccard}, Epoch: {e + 1}')
        wandb.log(
                  {'epoch': e,
                   "tr_loss": np.mean(tl), 
                   'vl loss': np.mean(l),
                   'jaccad': jaccard
                
                   }
                  )
        
        if jaccard > best_jaccard:
            print('\n')
            print(f'Best jaccard save: {jaccard}, prev: {best_jaccard}')
            type_model = config['type_model']
            torch.save(model.state_dict(), f'/content/drive/MyDrive/Coleridge_Initiative/model/model_{MODEL}_{type_model}_{EPOCHS}_{BATCH}_{FRAC}_{make_current_date()}.pth')
            best_jaccard = jaccard
            # pass

    # https://pytorch.org/tutorials/beginner/saving_loading_models.html    
    # torch.save(model.state_dict(), f'/content/drive/MyDrive/Coleridge_Initiative/model/model_{MODEL}_{EPOCHS}_{BATCH}_{FRAC}_{make_current_date()}.pth')
    wandb.finish()
    torch.cuda.empty_cache()    
    # return l,t, j, o

In [ ]:
df.drop_duplicates(subset=['id']).sample(frac = 0.1, random_state = 13).reset_index(drop=True).shape

(1396, 3)

In [ ]:
# l, t, j, o =
# %%time
run(config)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Valid loss: 0.4172400236129761, Jaccard Loss: 0.8069206096652397, Epoch: 1:   0%|          | 0/6 [14:23<?, ?it/s]



Best jaccard save: 0.8069206096652397, prev: 0


Valid loss: 0.39969244599342346, Jaccard Loss: 0.8078605121564549, Epoch: 2:  17%|█▋        | 1/6 [29:29<1:12:01, 864.21s/it]



Best jaccard save: 0.8078605121564549, prev: 0.8069206096652397


Valid loss: 0.38450759649276733, Jaccard Loss: 0.8136352730624806, Epoch: 3:  33%|███▎      | 2/6 [44:29<58:28, 877.14s/it]



Best jaccard save: 0.8136352730624806, prev: 0.8078605121564549


Valid loss: 0.40780121088027954, Jaccard Loss: 0.8277392875841563, Epoch: 4:  50%|█████     | 3/6 [59:14<44:10, 883.67s/it]



Best jaccard save: 0.8277392875841563, prev: 0.8136352730624806


Valid loss: 0.43061497807502747, Jaccard Loss: 0.8317157252718113, Epoch: 5:  67%|██████▋   | 4/6 [1:14:20<29:28, 884.24s/it]



Best jaccard save: 0.8317157252718113, prev: 0.8277392875841563


In [ ]:
# config={
   
#     "learning_rate": 0.0001,
#     'tex_max_len': 396,
#     'asw_max_len': 12,
#     'batch_size' : BATCH,
# }

# for txl in [332, 364, 396, 428, 460, 492, 524]:
# for lr in [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]:
# for lr in [0.01, 0.05, 0.08, 0.001, 0.005, 0.008, 0.0001]:
# for lr in [0.0008, 0.00001, 0.00003]:
# for txl in [396, 492, 524]:

# config={
#     'type_model': 'summary',
#     "learning_rate": 0.001,
#     "architecture": "T5",
#     'model': 't5-small', #'t5-base', # t5-small
#     "dataset": "Coleridge Initiative ",
#     'tex_max_len': 512,
#     'summar_max_len': 18,
#     'batch_size' : BATCH,
#     'epoch':EPOCHS,
#     'device': 'cuda'
# }

# for sml in [8, 12, 14, 18, 20, 24, 32]:
#     config['learning_rate'] = 0.0003
#     config['summar_max_len'] = sml
#     wandb.init(project="ci",
#            config=config,
#            name =  f"{config['architecture']}_{config['model']}_{config['type_model']}"+\
#                    f"_LR:{config['learning_rate']}_BATCH:{BATCH}_FRAC_DATA:{FRAC}" +\
#                    f"_summ_len{config['summar_max_len']}"
#            )
#     config_w = wandb.config    
#     run(config)

In [ ]:
# https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.TFPreTrainedModel.generate
# https://huggingface.co/blog/how-to-generate